In [67]:
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
import urllib
from sqlalchemy import text

tqdm.pandas()

Connect to Azure

In [68]:

conn_string_odbc="Driver={ODBC Driver 18 for SQL Server};Server=tcp:smartspace.database.windows.net,1433;Database=connectionspace;Uid=stats170-G6;Pwd=cro-r5sweDlVay5t=eta;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
params = urllib.parse.quote_plus(conn_string_odbc)
conn_str_formatted = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str_formatted)

Fetch Tables

In [69]:
# fetch table names
conn = engine.connect()

query = text("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
result = conn.execute(query)
for row in result:
    print(row)

conn.close()

('connectionspace', 'dbo', 'region_to_space', 'BASE TABLE')
('connectionspace', 'dbo', 'processed_data_merged', 'BASE TABLE')
('connectionspace', 'dbo', 'dbh_space', 'BASE TABLE')
('connectionspace', 'dbo', 'space_groups', 'BASE TABLE')
('connectionspace', 'dbo', 'space_hierarchy', 'BASE TABLE')
('connectionspace', 'dbo', 'Processed_Data', 'BASE TABLE')
('connectionspace', 'dbo', 'sensor', 'BASE TABLE')
('connectionspace', 'dbo', 'space_types', 'BASE TABLE')


In [70]:
# fetch region_to_space table
conn = engine.connect()

query = text("SELECT * FROM region_to_space")
results = conn.execute(query)

rows = results.fetchall()
cols = results.keys()
df_region_to_space = pd.DataFrame(rows, columns=cols)

conn.close()

print(df_region_to_space.head())

      sensor_name                                       rooms
0  3141-clwa-1100                    1100,1300,1407,1403,1406
1  3141-clwa-1200                              1100,1200,1300
2  3141-clwa-1300                         1100,1200,1300,1403
3  3141-clwa-1412  1403,1406,1428,1413,1300,1412,1420,1100_L1
4  3141-clwa-1420          1412,1420,1422,1407,1403,1406,1428


In [71]:
# Fetch space_groups table
conn = engine.connect()

query = text("SELECT * FROM space_groups")
results = conn.execute(query)

rows = results.fetchall()
cols = results.keys()
df_space_groups = pd.DataFrame(rows, columns=cols)

conn.close()

print(df_space_groups.head())

   location_id location_name location_type  parent_space_id research_group  \
0            1          1407           FAC              339           None   
1            2       1100_L1         CLASS              339           None   
2            3          1403          None              339           None   
3            4          1100         CLASS              339           None   
4            5          1434    ELECTRICAL              339           None   

  occupant research_areas  
0     None                 
1     None                 
2     None                 
3     None                 
4     None                 


Merging Space Tables

In [72]:
# Create list of rooms, instead of (rooms string separated by commas)
split_rooms = df_region_to_space['rooms'].str.split(',')

# New Dataframe for Rooms to Sensor 1:1
df_room_to_sensors = pd.DataFrame({
    'room': [room for rooms in split_rooms for room in rooms],
    'sensors': df_region_to_space['sensor_name'].repeat(split_rooms.apply(len))
})


# Group by room and convert individual sensors into list
df_room_to_sensors_grouped = df_room_to_sensors.groupby('room')['sensors'].agg(list).reset_index()

# Sort the DataFrame by 'room' column in ascending order
df_room_to_sensors_sorted = df_room_to_sensors_grouped.sort_values(by='room')

print(df_room_to_sensors_sorted)

        room                                            sensors
0       1100  [3141-clwa-1100, 3141-clwa-1200, 3141-clwa-130...
1    1100_L1                                   [3141-clwa-1412]
2       1200  [3141-clwa-1200, 3141-clwa-1300, 3141-clwb-120...
3       1300  [3141-clwa-1100, 3141-clwa-1200, 3141-clwa-130...
4       1403  [3141-clwa-1100, 3141-clwa-1300, 3141-clwa-141...
..       ...                                                ...
301     6212   [3146-clwa-6029, 3146-clwa-6217, 3146-clwa-6219]
302     6213                   [3146-clwa-6217, 3146-clwa-6219]
303     6215                   [3146-clwa-6217, 3146-clwa-6219]
304     6218   [3146-clwa-6011, 3146-clwa-6217, 3146-clwa-6219]
305     6219   [3146-clwa-6011, 3146-clwa-6217, 3146-clwa-6219]

[306 rows x 2 columns]


In [73]:
# Merge space columns into 1 resulting table
df_space_hierarchy = df_room_to_sensors_sorted.copy()

In [74]:
# Create floor column
df_space_hierarchy['floor'] = df_space_hierarchy['room'].str[0]

print(df_space_hierarchy)

        room                                            sensors floor
0       1100  [3141-clwa-1100, 3141-clwa-1200, 3141-clwa-130...     1
1    1100_L1                                   [3141-clwa-1412]     1
2       1200  [3141-clwa-1200, 3141-clwa-1300, 3141-clwb-120...     1
3       1300  [3141-clwa-1100, 3141-clwa-1200, 3141-clwa-130...     1
4       1403  [3141-clwa-1100, 3141-clwa-1300, 3141-clwa-141...     1
..       ...                                                ...   ...
301     6212   [3146-clwa-6029, 3146-clwa-6217, 3146-clwa-6219]     6
302     6213                   [3146-clwa-6217, 3146-clwa-6219]     6
303     6215                   [3146-clwa-6217, 3146-clwa-6219]     6
304     6218   [3146-clwa-6011, 3146-clwa-6217, 3146-clwa-6219]     6
305     6219   [3146-clwa-6011, 3146-clwa-6217, 3146-clwa-6219]     6

[306 rows x 3 columns]


In [75]:
# Check duplicate locations
duplicate_locations = df_space_groups[df_space_groups.duplicated(subset=['location_name'], keep=False)]

print(df_space_groups)

     location_id location_name location_type  parent_space_id research_group  \
0              1          1407           FAC              339           None   
1              2       1100_L1         CLASS              339           None   
2              3          1403          None              339           None   
3              4          1100         CLASS              339           None   
4              5          1434    ELECTRICAL              339           None   
..           ...           ...           ...              ...            ...   
339          340        Floor2          None              345           None   
340          341        Floor3          None              345           None   
341          342        Floor4          None              345           None   
342          343        Floor5          None              345           None   
343          344        Floor6          None              345           None   

    occupant research_areas  
0       N

In [76]:
# Remove the second occurrence of duplicates based on 'location_name'
df_space_groups = df_space_groups.drop_duplicates(subset=['location_name'], keep='first')

# Filter space_groups
df_space_groups_filtered = df_space_groups[['location_name','location_id', 'location_type']]

# Merge on room
df_space_hierarchy = df_space_hierarchy.join(df_space_groups_filtered.set_index('location_name'), on='room')

print(df_space_hierarchy)

        room                                            sensors floor  \
0       1100  [3141-clwa-1100, 3141-clwa-1200, 3141-clwa-130...     1   
1    1100_L1                                   [3141-clwa-1412]     1   
2       1200  [3141-clwa-1200, 3141-clwa-1300, 3141-clwb-120...     1   
3       1300  [3141-clwa-1100, 3141-clwa-1200, 3141-clwa-130...     1   
4       1403  [3141-clwa-1100, 3141-clwa-1300, 3141-clwa-141...     1   
..       ...                                                ...   ...   
301     6212   [3146-clwa-6029, 3146-clwa-6217, 3146-clwa-6219]     6   
302     6213                   [3146-clwa-6217, 3146-clwa-6219]     6   
303     6215                   [3146-clwa-6217, 3146-clwa-6219]     6   
304     6218   [3146-clwa-6011, 3146-clwa-6217, 3146-clwa-6219]     6   
305     6219   [3146-clwa-6011, 3146-clwa-6217, 3146-clwa-6219]     6   

     location_id location_type  
0            4.0         CLASS  
1            2.0         CLASS  
2           10.0        

In [77]:
# # Add table into Azure
# df_space_hierarchy.to_sql('space_hierarchy', engine, if_exists='replace', index=False)

# print("Table created successfully.")